In [1]:
pip install bs4

  Using cached bs4-0.0.1.tar.gz (1.1 kB)
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1277 sha256=49e6ef5e5ca4c66e15fa0ac5f8fb560918e5afc4bc0f024e564446b64993f923
  Stored in directory: c:\users\rush2\appdata\local\pip\cache\wheels\75\78\21\68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
Successfully built bs4
Note: you may need to restart the kernel to use updated packages.


In [2]:
#importing required libraries
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

## Extracting features of 2022 movies from Wikipedia

In [3]:
#adding wikipedia's link
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2022"

In [5]:
#using Beautiful soup for web scraping to extract movies of 2022 from wikipedia
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [6]:
tables = soup.find_all('table',class_='wikitable sortable')

In [7]:
len(tables)

4

In [8]:
type(tables[0])

bs4.element.Tag

In [9]:
#-----------avoided "ValueError in the fourth line: invalid literal for int() with base 10: '1"'------------
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0]  

In [10]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [11]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,JANUARY,7.0,The 355,Universal Pictures / Freckle Films / FilmNatio...,Simon Kinberg (director/screenplay); Theresa R...,[2],NaN
1,JANUARY,7.0,The Legend of La Llorona,Saban Films,Patricia Harris Seeley (director/screenplay); ...,[3],NaN
2,JANUARY,7.0,The Commando,Saban Films,Asif Akbar (director); Koji Steven Sakai (scre...,[4],NaN
3,JANUARY,14.0,Scream,Paramount Pictures / Spyglass Media Group,"Matt Bettinelli-Olpin, Tyler Gillett (director...",[5],NaN
4,JANUARY,14.0,Hotel Transylvania: Transformania,Amazon Studios / Columbia Pictures / Sony Pict...,"Jennifer Kluska, Derek Drymon (directors); Amo...",[6],NaN
...,...,...,...,...,...,...,...
190,DECEMBER,21.0,Puss in Boots: The Last Wish,Universal Pictures / DreamWorks Animation,"Joel Crawford (director); Antonio Banderas, Sa...",NaN,[162]
191,DECEMBER,21.0,I Wanna Dance with Somebody,TriStar Pictures / Black Label Media,Kasi Lemmons (director); Anthony McCarten (scr...,NaN,[53]
192,DECEMBER,25.0,Babylon,Paramount Pictures,Damien Chazelle (director/screenplay); Brad Pi...,NaN,[163]
193,DECEMBER,25.0,A Man Called Otto,Columbia Pictures / Playtone / SF Studios,"Marc Forster (director), David Magee (screenpl...",NaN,[53]


In [13]:
#---taking 2 features from the dataset and dropping the rest of the features-----
df_2022 = df[['Title','Cast and crew']]

In [14]:
df_2022

,Title,Cast and crew
0,The 355,Simon Kinberg (director/screenplay); Theresa R...
1,The Legend of La Llorona,Patricia Harris Seeley (director/screenplay); ...
2,The Commando,Asif Akbar (director); Koji Steven Sakai (scre...
3,Scream,"Matt Bettinelli-Olpin, Tyler Gillett (director..."
4,Hotel Transylvania: Transformania,"Jennifer Kluska, Derek Drymon (directors); Amo..."
...,...,...
190,Puss in Boots: The Last Wish,"Joel Crawford (director); Antonio Banderas, Sa..."
191,I Wanna Dance with Somebody,Kasi Lemmons (director); Anthony McCarten (scr...
192,Babylon,Damien Chazelle (director/screenplay); Brad Pi...
193,A Man Called Otto,"Marc Forster (director), David Magee (screenpl..."


In [15]:
!pip install tmdbv3api

In [16]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = '6d68a00f1b47054bb4ab00965d3e1416'

In [17]:
#defining get_genre function to extract genres of 2022's movies using its tmdb id and tmdb key
from tmdbv3api import Movie
tmdb_movie = Movie() 
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
      data_json = response.json()
      if data_json['genres']:
          genre_str = " " 
          for i in range(0,len(data_json['genres'])):
              genres.append(data_json['genres'][i]['name'])
          return genre_str.join(genres)
      else:
          return np.NaN

In [18]:
#mapping 2022's movies to its respective genre
df_2022['genres'] = df_2022['Title'].map(lambda x: get_genre(str(x)))

<ipython-input-18-b303003edd4f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['genres'] = df_2022['Title'].map(lambda x: get_genre(str(x)))


In [19]:
df_2022

,Title,Cast and crew,genres
0,The 355,Simon Kinberg (director/screenplay); Theresa R...,Action Thriller
1,The Legend of La Llorona,Patricia Harris Seeley (director/screenplay); ...,Horror Thriller
2,The Commando,Asif Akbar (director); Koji Steven Sakai (scre...,Action Crime Thriller
3,Scream,"Matt Bettinelli-Olpin, Tyler Gillett (director...",Horror Mystery Thriller
4,Hotel Transylvania: Transformania,"Jennifer Kluska, Derek Drymon (directors); Amo...",Animation Family Fantasy Comedy Adventure
...,...,...,...
190,Puss in Boots: The Last Wish,"Joel Crawford (director); Antonio Banderas, Sa...",Animation Adventure Comedy Family Fantasy
191,I Wanna Dance with Somebody,Kasi Lemmons (director); Anthony McCarten (scr...,Drama History
192,Babylon,Damien Chazelle (director/screenplay); Brad Pi...,Action Adventure Science Fiction Thriller
193,A Man Called Otto,"Marc Forster (director), David Magee (screenpl...",Drama Comedy


In [20]:
#function to get 1st actor's information
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [22]:
df_2022['actor_1_name'] = df_2022['Cast and crew'].map(lambda x: get_actor1(str(x)))

<ipython-input-22-e67b31464e65>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['actor_1_name'] = df_2022['Cast and crew'].map(lambda x: get_actor1(str(x)))


In [23]:
#function to get 2nd actor's information
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [24]:
df_2022['actor_2_name'] = df_2022['Cast and crew'].map(lambda x: get_actor2(str(x)))

<ipython-input-24-d3d5c35d26fc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['actor_2_name'] = df_2022['Cast and crew'].map(lambda x: get_actor2(str(x)))


In [25]:
#function to get 3rd actor's information
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [26]:
df_2022['actor_3_name'] = df_2022['Cast and crew'].map(lambda x: get_actor3(str(x)))

<ipython-input-26-0c65546c30a0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['actor_3_name'] = df_2022['Cast and crew'].map(lambda x: get_actor3(str(x)))


In [27]:
#function to get director's information
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [28]:
df_2022['director_name'] = df_2022['Cast and crew'].map(lambda x: get_director(str(x)))

In [29]:
df_2022

,Title,Cast and crew,genres,actor_1_name,actor_2_name,actor_3_name,director_name
0,The 355,Simon Kinberg (director/screenplay); Theresa R...,Action Thriller,Jessica Chastain,Lupita Nyong'o,Penélope Cruz,Simon Kinberg
1,The Legend of La Llorona,Patricia Harris Seeley (director/screenplay); ...,Horror Thriller,Autumn Reeser,Antonio Cupo,Danny Trejo,Patricia Harris Seeley
2,The Commando,Asif Akbar (director); Koji Steven Sakai (scre...,Action Crime Thriller,Mickey Rourke,Michael Jai White,NaN,Asif Akbar
3,Scream,"Matt Bettinelli-Olpin, Tyler Gillett (director...",Horror Mystery Thriller,Melissa Barrera,Mason Gooding,Jenna Ortega,"Matt Bettinelli-Olpin, Tyler Gillett"
4,Hotel Transylvania: Transformania,"Jennifer Kluska, Derek Drymon (directors); Amo...",Animation Family Fantasy Comedy Adventure,Andy Samberg,Selena Gomez,Kathryn Hahn,"Jennifer Kluska, Derek Drymon"
...,...,...,...,...,...,...,...
190,Puss in Boots: The Last Wish,"Joel Crawford (director); Antonio Banderas, Sa...",Animation Adventure Comedy Family Fantasy,Joel Crawford (director); Antonio Banderas,Salma Hayek,Harvey Guillén,Joel Crawford
191,I Wanna Dance with Somebody,Kasi Lemmons (director); Anthony McCarten (scr...,Drama History,Naomi Ackie,Ashton Sanders,Stanley Tucci,Kasi Lemmons
192,Babylon,Damien Chazelle (director/screenplay); Brad Pi...,Action Adventure Science Fiction Thriller,Brad Pitt,Margot Robbie,Tobey Maguire,Damien Chazelle
193,A Man Called Otto,"Marc Forster (director), David Magee (screenpl...",Drama Comedy,Tom Hanks,Manuel Garcia-Rulfo,NaN,Marc Forster


In [30]:
df_2022 = df_2022.rename(columns={'Title':'movie_title'})

In [31]:
new_df22 = df_2022.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [32]:
new_df22

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Simon Kinberg,Jessica Chastain,Lupita Nyong'o,Penélope Cruz,Action Thriller,The 355
1,Patricia Harris Seeley,Autumn Reeser,Antonio Cupo,Danny Trejo,Horror Thriller,The Legend of La Llorona
2,Asif Akbar,Mickey Rourke,Michael Jai White,NaN,Action Crime Thriller,The Commando
3,"Matt Bettinelli-Olpin, Tyler Gillett",Melissa Barrera,Mason Gooding,Jenna Ortega,Horror Mystery Thriller,Scream
4,"Jennifer Kluska, Derek Drymon",Andy Samberg,Selena Gomez,Kathryn Hahn,Animation Family Fantasy Comedy Adventure,Hotel Transylvania: Transformania
...,...,...,...,...,...,...
190,Joel Crawford,Joel Crawford (director); Antonio Banderas,Salma Hayek,Harvey Guillén,Animation Adventure Comedy Family Fantasy,Puss in Boots: The Last Wish
191,Kasi Lemmons,Naomi Ackie,Ashton Sanders,Stanley Tucci,Drama History,I Wanna Dance with Somebody
192,Damien Chazelle,Brad Pitt,Margot Robbie,Tobey Maguire,Action Adventure Science Fiction Thriller,Babylon
193,Marc Forster,Tom Hanks,Manuel Garcia-Rulfo,NaN,Drama Comedy,A Man Called Otto


In [33]:
new_df22['comb'] = new_df22['actor_1_name'] + ' ' + new_df22['actor_2_name'] + ' '+ new_df22['actor_3_name'] + ' '+ new_df22['director_name'] +' ' + new_df22['genres']

In [34]:
#checking null values
new_df22.isna().sum()

director_name     0
actor_1_name      0
actor_2_name      6
actor_3_name     20
genres            1
movie_title       2
comb             20
dtype: int64

In [35]:
#------dropping null values-----
new_df22 = new_df22.dropna(how='any')

In [36]:
new_df22.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [37]:
new_df22['movie_title'] = new_df22['movie_title'].str.lower()

<ipython-input-37-bd701421fe8d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df22['movie_title'] = new_df22['movie_title'].str.lower()


In [38]:
new_df22

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Simon Kinberg,Jessica Chastain,Lupita Nyong'o,Penélope Cruz,Action Thriller,the 355,Jessica Chastain Lupita Nyong'o Penélope Cruz ...
1,Patricia Harris Seeley,Autumn Reeser,Antonio Cupo,Danny Trejo,Horror Thriller,the legend of la llorona,Autumn Reeser Antonio Cupo Danny Trejo Patrici...
3,"Matt Bettinelli-Olpin, Tyler Gillett",Melissa Barrera,Mason Gooding,Jenna Ortega,Horror Mystery Thriller,scream,Melissa Barrera Mason Gooding Jenna Ortega Mat...
4,"Jennifer Kluska, Derek Drymon",Andy Samberg,Selena Gomez,Kathryn Hahn,Animation Family Fantasy Comedy Adventure,hotel transylvania: transformania,Andy Samberg Selena Gomez Kathryn Hahn Jennife...
5,Luis Prieto,Cameron Monaghan,Frank Grillo,Lilly Krug,Thriller,shattered,Cameron Monaghan Frank Grillo Lilly Krug Luis ...
...,...,...,...,...,...,...,...
188,James Cameron,Sam Worthington,Zoe Saldana,Stephen Lang,Action Adventure Fantasy Science Fiction,avatar: the way of water,Sam Worthington Zoe Saldana Stephen Lang James...
189,David F. Sandberg,Zachary Levi,Asher Angel,Jack Dylan Grazer,Comedy Action Fantasy,shazam! fury of the gods,Zachary Levi Asher Angel Jack Dylan Grazer Dav...
190,Joel Crawford,Joel Crawford (director); Antonio Banderas,Salma Hayek,Harvey Guillén,Animation Adventure Comedy Family Fantasy,puss in boots: the last wish,Joel Crawford (director); Antonio Banderas Sal...
191,Kasi Lemmons,Naomi Ackie,Ashton Sanders,Stanley Tucci,Drama History,i wanna dance with somebody,Naomi Ackie Ashton Sanders Stanley Tucci Kasi ...


In [40]:
final_df = pd.read_csv('C:/Users/rush2/Documents/Movieflix/python_notebooks_recommendation_engine/main_data_till_2021.csv')

In [41]:
final_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
6449,Matthew Vaughn,Ralph Fiennes,Gemma Arterton,Rhys Ifans,Action Adventure Thriller War Mystery,the king's man,Ralph Fiennes Gemma Arterton Rhys Ifans Matthe...
6450,Joel Coen,Denzel Washington,Frances McDormand,Bertie Carvel,Drama War,the tragedy of macbeth,Denzel Washington Frances McDormand Bertie Car...
6451,Denzel Washington,Michael B. Jordan,Chanté Adams,Jalon Christian,Drama Romance,a journal for jordan,Michael B. Jordan Chanté Adams Jalon Christian...
6452,Erwin brothers,Zachary Levi,Anna Paquin,Dennis Quaid,Drama,american underdog,Zachary Levi Anna Paquin Dennis Quaid Erwin br...


In [42]:
#appending the newly extracted 2022's movies to the older data
final_df = final_df.append(new_df22,ignore_index=True)

In [43]:
final_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
6624,James Cameron,Sam Worthington,Zoe Saldana,Stephen Lang,Action Adventure Fantasy Science Fiction,avatar: the way of water,Sam Worthington Zoe Saldana Stephen Lang James...
6625,David F. Sandberg,Zachary Levi,Asher Angel,Jack Dylan Grazer,Comedy Action Fantasy,shazam! fury of the gods,Zachary Levi Asher Angel Jack Dylan Grazer Dav...
6626,Joel Crawford,Joel Crawford (director); Antonio Banderas,Salma Hayek,Harvey Guillén,Animation Adventure Comedy Family Fantasy,puss in boots: the last wish,Joel Crawford (director); Antonio Banderas Sal...
6627,Kasi Lemmons,Naomi Ackie,Ashton Sanders,Stanley Tucci,Drama History,i wanna dance with somebody,Naomi Ackie Ashton Sanders Stanley Tucci Kasi ...


In [44]:
final_df.to_csv('main_data_till_2022.csv',index=False)